In [1]:
import sys,os
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

In [2]:
stemmer = SnowballStemmer('english')
stopW = stopwords.words('english')
emoji_pattern = re.compile("["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    u"\U00002702-\U000027B0"
    u"\U000024C2-\U0001F251"
"]+", flags=re.UNICODE)
stem_map={}

In [3]:
def load_data(filename):
    n = ['id', 'text','HS','TR','AG']
    given_data = pd.read_csv(filename, sep='\t',error_bad_lines=False, names=n, usecols=['text','HS','TR','AG'], skiprows=1)
    raw_data = given_data['text'].values
    labels_TR = list(map(int,given_data['TR'].values))
    labels_AG = list(map(int,given_data['AG'].values))
    labels_HS = list(map(int,given_data['HS'].values))
    
    data = [preprocess(tweet) for tweet in raw_data]
    X = []
    y_AG = []
    y_TR = []
    
    for index,word in enumerate(labels_HS):
        if word:
            X.append(data[index])
            y_AG.append(labels_AG[index]) 
            y_TR.append(labels_TR[index])

    
    return X, y_AG, y_TR

In [4]:
def preprocess(tweet):
    # ' '.join([word for word in tweet.spilt() ])
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL', tweet)
    tweet = re.sub('@[^\s]+','USER', tweet)
    tweet = tweet.replace("ё", "е")
    tweet = re.sub('[^a-zA-Zа-яА-Я1-9]+', ' ', tweet)
    tweet = re.sub(' +',' ', tweet)
    tweet = emoji_pattern.sub(r'', tweet)

    stemmed_text_token=[]
    tokens = tweet.split(' ')
    for token in tokens:
        if token=='':
            continue
        elif token=='USER' or token=='URL': 
            stemmed_text_token.append(token)
        # if token not in stopW:
        #Need to check performance with and without stopwords.
        else:
            a=stem_map.get(token,0)
            if a==0:
                a=stemmer.stem(token)
                stem_map[token]=a
            stemmed_text_token.append(a)
    return ' '.join(stemmed_text_token)


In [5]:
def classifier(data, labels):
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.01, random_state=324)
    clf = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer(use_idf=False,norm='l2')),
                     ('clf', LogisticRegression(penalty = 'l2'))])

    parameters = {
        'vect__ngram_range': [(1, 1), (1, 2),(2,2)],
        'tfidf__use_idf': (True, False),
        'tfidf__norm': ('l1', 'l2'),
        'clf__alpha': [10,1, 1e-1, 1e-2,1e-3]
    }
#     clf = GridSearchCV(clf, parameters, cv=5, iid=False, n_jobs=-1)

    clf.fit(X_train,y_train)
    prediction = clf.predict(X_test)
#     accuracy_score = text_clf.score(X_test,y_test)
    return clf

In [6]:
def test(X, Y, clf):
    print(classification_report(Y, clf.predict(X), digits=4))

In [11]:
def main():
    training_file = 'train_en.tsv'
    testing_file = 'dev_en.tsv'
    
    X_train, y_AG_train, y_TR_train = load_data(training_file)
    X_test, y_AG_test, y_TR_test = load_data(testing_file)
    
    clf_aggresive = classifier(X_train, y_AG_train)
    clf_target = classifier(X_train, y_TR_train)
    
    print("Results aggresiveness on training data")
    test(clf_aggresive, X_train, y_AG_train)

    print("Results aggresiveness on test data")
    test(clf_aggresive, X_test, y_AG_test)

    print("Results targeting on training data")
    test(clf_target, X_train, y_TR_train)

    print("Results targeting on test data")
    test(clf_target, X_test, y_TR_test)


In [12]:
if __name__=='__main__':
    main()

/home/tanujgarg/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Results aggresiveness on training data


/home/tanujgarg/.local/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


AttributeError: 'list' object has no attribute 'predict'